In [19]:
import pandas as pd
import numpy as np
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.utils import resample
import statsmodels.api as sm
from scipy import stats
from sklearn.linear_model import LogisticRegression
from statsmodels.stats.diagnostic import het_breuschpagan
from statsmodels.stats.stattools import durbin_watson

## Data

In [2]:
df = pd.read_csv('data/cdv.csv')
print(df.columns.tolist())

['X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X7', 'X8', 'X9', 'X10', 'Y']


In [3]:
df.head(2)

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,Y
0,1.1,0,1,28.44,134,3.54,4.32,55.7,0,0,1
1,1.3,1,1,34.13,126,5.87,3.95,53.1,0,1,1


In [4]:
df.describe()

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,Y
count,21.000000,21.000000,21.000000,21.000000,21.000000,21.000000,21.000000,21.000000,21.000000,21.000000,21.000000
mean,3.666667,0.380952,0.380952,31.935238,133.761905,4.633333,4.604286,67.004762,0.476190,0.666667,1.809524
std,1.470147,0.497613,0.497613,7.573592,27.933680,2.248318,0.531522,8.520650,0.511766,0.483046,0.749603
min,1.100000,0.000000,0.000000,20.780000,96.000000,1.380000,3.900000,52.500000,0.000000,0.000000,1.000000
25%,2.700000,0.000000,0.000000,25.220000,118.000000,2.690000,4.180000,59.400000,0.000000,0.000000,1.000000
50%,3.600000,0.000000,0.000000,30.880000,128.000000,4.400000,4.520000,69.100000,0.000000,1.000000,2.000000
75%,4.600000,1.000000,1.000000,36.830000,134.000000,5.870000,4.860000,73.100000,1.000000,1.000000,2.000000
max,6.300000,1.000000,1.000000,46.760000,200.000000,9.930000,5.630000,77.200000,1.000000,1.000000,3.000000


In [5]:
X = df.drop(columns=['Y'])  
y = df['Y']

n = len(y)
p = df.shape[1]
k = p - 1

| Variabel | Keterangan                              | Skala        |
|----------|----------------------------------------|-------------|
| X1       | Waktu tahan hidup pasien                | Rasio       |
| X2       | Jenis kelamin pasien                    | Nominal     |
| X3       | Intensitas merokok                      | Nominal     |
| X4       | Indeks massa tubuh                      | Rasio       |
| X5       | Tekanan darah sistolik (mmHg)           | Rasio       |
| X6       | Logaritme rasio albumin dan kreatinin urin | Rasio    |
| X7       | Logaritme trigliserida                  | Rasio       |
| X8       | Umur pasien                             | Rasio       |
| X9       | Status hipertensi                        | Nominal     |
| X10      | Status diabetes                          | Nominal     |
| Y        | Jenis penyakit kardiovaskular           | Nominal     |
|          | 1: stroke                               |             |
|          | 2: coronary heart disease               |             |
|          | 3: angina                               |             |

In [6]:
print(y.value_counts()) 

Y
2    9
1    8
3    4
Name: count, dtype: int64


## Regresi Logistik Multinomial

### Uji Multikolinearitas

In [7]:
X_const = add_constant(X)

vif_data = pd.DataFrame()
vif_data["Variable"] = X_const.columns
vif_data["VIF"] = [variance_inflation_factor(X_const.values, i) for i in range(X_const.shape[1])]
print(vif_data)

   Variable         VIF
0     const  310.528334
1        X1    1.659555
2        X2    1.321321
3        X3    1.370177
4        X4    2.297288
5        X5    2.571857
6        X6    2.002548
7        X7    1.737764
8        X8    2.139399
9        X9    2.152826
10      X10    1.870077


- **H₀**: \($ \beta_{xy} = 0 $\) (Tidak ada multikolinearitas, tidak ada variabel yang berikatan terlalu kuat)
- **H₁**: \($ \beta_{xy} \neq 0 $\) (Variabel bebas \($ x $\) berpengaruh signifikan terhadap variabel terikat \($ z $\))

### Bangun Model

In [8]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
logit_model = LogisticRegression(solver='lbfgs', class_weight='balanced', max_iter=1000)
logit_model.fit(X_scaled, y)
y_pred = logit_model.predict(X_scaled)

coef_df = pd.DataFrame(logit_model.coef_, columns=X.columns)
coef_df

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10
0,-0.690025,-0.282688,0.133924,0.201086,0.713026,0.083922,-0.301407,-0.216271,-0.779790,-0.343182
1,0.224926,-0.140859,0.399372,0.012832,-0.797438,-0.558139,0.240112,0.356680,-0.240599,0.894988
2,0.465099,0.423547,-0.533296,-0.213918,0.084412,0.474217,0.061295,-0.140409,1.020389,-0.551806


# Koefisien Model Regresi Logistik Multinomial

Setelah model regresi logistik multinomial di-fit, kita memperoleh koefisien untuk setiap fitur pada masing-masing kelas. Berikut adalah koefisien yang dihasilkan:

### Koefisien untuk Kelas 1 (Coronary Heart Disease):
$$
\log \left( \frac{P(Y = 1)}{P(Y = 0)} \right) = -0.690025 X_1 - 0.282688 X_2 + 0.133924 X_3 + 0.201086 X_4 + 0.713026 X_5 + 0.083922 X_6 - 0.301407 X_7 - 0.216271 X_8 - 0.779790 X_9 - 0.343182 X_{10}
$$

### Koefisien untuk Kelas 2 (Stroke):
$$
\log \left( \frac{P(Y = 2)}{P(Y = 0)} \right) = 0.224926 X_1 - 0.140859 X_2 + 0.399372 X_3 + 0.012832 X_4 - 0.797438 X_5 - 0.558139 X_6 + 0.240112 X_7 + 0.356680 X_8 - 0.240599 X_9 + 0.894988 X_{10}
$$
### Koefisien untuk Kelas 3 (Angina):
$$
\log \left( \frac{P(Y = 3)}{P(Y = 0)} \right) = 0.465099 X_1 + 0.423547 X_2 - 0.533296 X_3 - 0.213918 X_4 + 0.084412 X_5 + 0.474217 X_6 + 0.061295 X_7 - 0.140409 X_8 + 1.020389 X_9 - 0.551806 X_{10}
$$

Dimana:
- \($ P(Y = k) $\) adalah probabilitas bahwa variabel respon \($ Y $\) adalah kelas \($ k $\).
- \($ X_1, X_2, \dots, X_{10} $\) adalah variabel prediktor (fitur).
- \($ P(Y = 0) $\) adalah probabilitas kelas referensi (biasanya kelas dengan indeks 0).

Dengan koefisien-koefisien ini, dapat dipahami bagaimana setiap fitur mempengaruhi kemungkinan terjadinya masing-masing jenis penyakit kardiovaskular. Koefisien yang positif menunjukkan peningkatan kemungkinan kelas tersebut, sementara koefisien negatif menunjukkan pengurangan kemungkinan kelas tersebut.


### Uji Simultan

In [9]:
y_prob = logit_model.predict_proba(X_scaled)
y_adj = y - 1 
llf = 0
for i in range(len(y_adj)):
    llf += np.log(y_prob[i, y_adj[i]])

llf_total = llf
print(f"Log-Likelihood Function: {llf_total}")

Log-Likelihood Function: -6.321157005828191


In [10]:
y_prob = logit_model.predict_proba(X_scaled)
y_int = np.array(y) - 1 
llf = np.sum(np.log(y_prob[np.arange(len(y)), y_int]))
print("Log-Likelihood Function (LLF):", llf)

Log-Likelihood Function (LLF): -6.321157005828192


- **H₀**: \($ \beta_1 = \beta_2 = \dots = \beta_n = 0 $\)  ;  \($ \beta_{xz} = 0 $\) (Tidak ada peubah prediktor (variabel bebas) yang memberikan pengaruh terhadap peubah respon)
- **H₁**: Ada minimal satu variabel bebas yang benar-benar berpengaruh.

### Uji Parsial

In [11]:
n_iterations = 1000
n_classes, n_features = logit_model.coef_.shape

coef_bootstrap = np.zeros((n_iterations, n_classes, n_features))

for i in range(n_iterations):
    X_resampled, y_resampled = resample(X_scaled, y, n_samples=len(y))
    logit_model.fit(X_resampled, y_resampled)
    coef_bootstrap[i] = logit_model.coef_

coef_se = np.std(coef_bootstrap, axis=0)
wald_stats = (logit_model.coef_ / coef_se) ** 2
p_values = 1 - stats.chi2.cdf(wald_stats, df=1)

print("Koefisien Model:\n", logit_model.coef_)
print("Standar Error (Bootstrap):\n", coef_se)
print("Statistik Wald:\n", wald_stats)
print("P-Value:\n", p_values)


Koefisien Model:
 [[-1.21674582 -0.33462181 -0.07055408  0.2372823   0.49780536 -0.38105127
  -0.23288937 -0.21971342 -0.40751687 -0.26814273]
 [ 0.5997715  -0.1491137   0.45964739 -0.33632919 -0.71231187 -0.1455539
   0.28889783  0.11673056 -0.40765708  0.79811263]
 [ 0.61697432  0.48373551 -0.38909331  0.09904689  0.21450651  0.52660516
  -0.05600845  0.10298287  0.81517395 -0.52996991]]
Standar Error (Bootstrap):
 [[0.31573643 0.25051856 0.23483348 0.24457217 0.35068942 0.30446875
  0.25417975 0.25761525 0.24981748 0.24307355]
 [0.26768492 0.25052503 0.23393084 0.25753304 0.2585541  0.22160079
  0.25105646 0.24175535 0.2238643  0.2379136 ]
 [0.19153066 0.28204637 0.17505268 0.33188076 0.27639614 0.22641082
  0.28818487 0.24457418 0.22997556 0.27365183]]
Statistik Wald:
 [[14.85081695  1.784139    0.09026601  0.94127523  2.01499419  1.56632316
   0.8394937   0.72739452  2.66100397  1.2169049 ]
 [ 5.0202325   0.35426875  3.86077625  1.70554519  7.58992547  0.43142476
   1.32417615  0.

- **H₀**: \($ \beta_z = 0 $\) (Peubah prediktor ke-\($ z $\) tidak memengaruhi nilai peubah respon dalam model)
- **H₁**: \($ \beta_z \neq 0 $\) (Peubah prediktor ke-\($ z $\) memengaruhi nilai peubah respon dalam model)

### Uji Kesesuaian Model

#### Uji Pearson Chi-Squared

In [12]:
cm = confusion_matrix(y, y_pred)
n_samples = X_scaled.shape[0]
expected_frequencies = np.outer(np.sum(cm, axis=1), np.sum(cm, axis=0)) / n_samples
pearson_chi_squared = np.sum((cm - expected_frequencies)**2 / expected_frequencies)

print(f"Pearson's Chi-squared statistic: {pearson_chi_squared}")

Pearson's Chi-squared statistic: 37.592592592592595


#### Log-Likelihood dan Deviance

In [13]:
log_likelihood_model = np.sum(np.log(y_prob[np.arange(len(y_adj)), y_adj]))
log_likelihood_null = np.sum(np.log(np.mean(y_prob, axis=0)[y_adj]))
deviance = -2 * (log_likelihood_model - log_likelihood_null)

print(f"Log-Likelihood Model: {log_likelihood_model}")
print(f"Log-Likelihood Null: {log_likelihood_null}")
print(f"Deviance: {deviance}")

Log-Likelihood Model: -6.321157005828192
Log-Likelihood Null: -22.05754336597345
Deviance: -31.47277272029052


#### Pseudo-R² (McFadden's Pseudo-R²)

In [14]:
pseudo_r2 = 1 - (log_likelihood_model / log_likelihood_null)
print(f"McFadden's Pseudo-R²: {pseudo_r2}")

McFadden's Pseudo-R²: 0.7134242512437095


### Asumsi Klasik

In [17]:
y_pred_prob = logit_model.predict_proba(X_scaled)
y_pred = np.argmax(y_pred_prob, axis=1)
residuals = (y_pred != y).astype(int)

#### Uji Homoskedastisitas (Breusch-Pagan Test)

In [20]:
X_const = sm.add_constant(X_scaled) 
bp_test = het_breuschpagan(residuals, X_const)

print("Uji Breusch-Pagan untuk Homoskedastisitas")
print("Lagrange Multiplier (LM) Stat:", bp_test[0])
print("p-value:", bp_test[1])

Uji Breusch-Pagan untuk Homoskedastisitas
Lagrange Multiplier (LM) Stat: 6.621765835266791
p-value: 0.760604663470878


* **H₀**: Tidak ada heteroskedastisitas (peubah error memiliki varians yang konstan, atau homoskedastisitas).

  (\$ \text{Var}(\epsilon_i) = \sigma^2 \quad \forall i
  $\)

* **H₁**: Terdapat heteroskedastisitas (peubah error memiliki varians yang tidak konstan).

  [
  \text{Var}(\epsilon_i) \neq \sigma^2 \quad \text{untuk beberapa nilai } i
  ]

#### Uji Autokorelasi Residual (Durbin-Watson Test)

In [21]:
dw_stat = durbin_watson(residuals)

print("\nDurbin-Watson Test untuk Autokorelasi Residual")
print("Statistik Durbin-Watson:", dw_stat)


Durbin-Watson Test untuk Autokorelasi Residual
Statistik Durbin-Watson: 0.21052631578947367
